## Step 1: Handling All Dataset Files

In [13]:
# --- 1. SETUP LIBRARIES AND DATA PATH ---
import pandas as pd
import numpy as np
import os
import glob

# IMPORTANT: Put all 8 of your CICIDS2017 CSV files into one folder.
# Then, update the path to that folder here.
DATA_DIR = 'D:/IoC-Free IDS using ML and NLP/data/raw/CICIDS2017/'

print(f"Starting data processing pipeline...")
print(f"Current time: {pd.Timestamp.now(tz='Asia/Kolkata').strftime('%Y-%m-%d %H:%M:%S %Z')}")

Starting data processing pipeline...
Current time: 2025-09-20 10:31:33 IST


In [ ]:
# --- 2. FIND AND PROCESS ALL CSV FILES IN A LOOP ---
# Find all CSV file paths in the specified directory

all_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))
print(f"\nFound {len(all_files)} CSV files to process in '{DATA_DIR}'.")

# This list will hold each cleaned DataFrame before we combine them

list_of_cleaned_dfs = []

for file_path in all_files:
    file_name = os.path.basename(file_path)
    print(f"--> Processing: {file_name}")
    
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # a. Clean leading/trailing spaces from column names
    df.columns = df.columns.str.strip()
    
    # b. Drop the IoC column ('Destination Port') for robust, behavioral detection
    if 'Destination Port' in df.columns:
        df.drop(columns = ['Destination Port'], inplace = True)
    
    # c. Handle potential duplicate column names by keeping the first instance
    df = df.loc[:, ~df.columns.duplicated()]

    # d. Clean dirty data: replace infinities with NaN, then drop all rows with any nulls
    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    df.dropna(inplace = True)
    
    # e. Add the fully cleaned DataFrame to our list
    list_of_cleaned_dfs.append(df)


Found 8 CSV files to process in 'D:/IoC-Free IDS using ML and NLP/data/raw/CICIDS2017/'.
--> Processing: Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
--> Processing: Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
--> Processing: Friday-WorkingHours-Morning.pcap_ISCX.csv
--> Processing: Monday-WorkingHours.pcap_ISCX.csv
--> Processing: Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
--> Processing: Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
--> Processing: Tuesday-WorkingHours.pcap_ISCX.csv
--> Processing: Wednesday-workingHours.pcap_ISCX.csv


In [16]:
# --- 3. COMBINE INTO A SINGLE MASTER DATAFRAME ---
print("\n--> Combining all processed files into a single master DataFrame...")
master_df = pd.concat(list_of_cleaned_dfs, ignore_index = True)
print("Combination complete!")


--> Combining all processed files into a single master DataFrame...
Combination complete!


In [17]:
# --- 4. FINALIZE & STANDARDIZE THE MASTER DATAFRAME ---
print("--> Finalizing the master DataFrame...")

# a. Standardize column names to snake_case (e.g., 'Flow Duration' -> 'flow_duration')
master_df.columns = master_df.columns.str.replace(' ', '_').str.lower()
print("    - Standardized column names to snake_case.")

# b. Remove any columns that have zero variance (i.e., the same value in every row)
numeric_cols = master_df.select_dtypes(include=np.number).columns
column_std = master_df[numeric_cols].std()
zero_std_cols = column_std[column_std == 0].index.tolist()

if zero_std_cols:
    master_df.drop(columns=zero_std_cols, inplace=True)
    print(f"    - Dropped {len(zero_std_cols)} zero-variance columns: {zero_std_cols}")
else:
    print("    - No zero-variance columns found.")

--> Finalizing the master DataFrame...
    - Standardized column names to snake_case.
    - Dropped 8 zero-variance columns: ['bwd_psh_flags', 'bwd_urg_flags', 'fwd_avg_bytes/bulk', 'fwd_avg_packets/bulk', 'fwd_avg_bulk_rate', 'bwd_avg_bytes/bulk', 'bwd_avg_packets/bulk', 'bwd_avg_bulk_rate']


In [19]:
# --- 5. VERIFY AND SAVE THE FINAL RESULT ---
print("\n--- Processing Complete! ---")
print(f"Final DataFrame Shape: {master_df.shape[0]:,} rows, {master_df.shape[1]} columns.")
print("\nFinal DataFrame Info:")
master_df.info()

# Display a sample of the new standardized column names
print("\nSample of new column names:")
print(master_df.columns[:5].tolist())

# Save the final, clean DataFrame to a fast-loading Parquet file
output_filename = 'D:/IoC-Free IDS using ML and NLP/data/processed/cicids2017_cleaned_standardized.parquet'
master_df.to_parquet(output_filename)
print(f"\n✅ Success! Cleaned data has been saved to '{output_filename}'.")


--- Processing Complete! ---
Final DataFrame Shape: 2,827,876 rows, 70 columns.

Final DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827876 entries, 0 to 2827875
Data columns (total 70 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   flow_duration                int64  
 1   total_fwd_packets            int64  
 2   total_backward_packets       int64  
 3   total_length_of_fwd_packets  int64  
 4   total_length_of_bwd_packets  int64  
 5   fwd_packet_length_max        int64  
 6   fwd_packet_length_min        int64  
 7   fwd_packet_length_mean       float64
 8   fwd_packet_length_std        float64
 9   bwd_packet_length_max        int64  
 10  bwd_packet_length_min        int64  
 11  bwd_packet_length_mean       float64
 12  bwd_packet_length_std        float64
 13  flow_bytes/s                 float64
 14  flow_packets/s               float64
 15  flow_iat_mean                float64
 16  flow_iat_std          

In [21]:
# The CSV file is generated for personal reference and out project will use the cleaned parquet as it is faster ans more efficient.

print("\n--- Processing Complete! ---")
print(f"Final DataFrame Shape: {master_df.shape[0]:,} rows, {master_df.shape[1]} columns.")
print("\nFinal DataFrame Info:")
master_df.info()

# Display a sample of the new standardized column names
print("\nSample of new column names:")
print(master_df.columns[:5].tolist())

# Save the final, clean DataFrame to a fast-loading Parquet file
output_filename = 'D:/IoC-Free IDS using ML and NLP/data/processed/cicids2017_cleaned_standardized.csv'
master_df.to_csv(output_filename)
print(f"\n✅ Success! Cleaned data has been saved to '{output_filename}'.")


--- Processing Complete! ---
Final DataFrame Shape: 2,827,876 rows, 70 columns.

Final DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2827876 entries, 0 to 2827875
Data columns (total 70 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   flow_duration                int64  
 1   total_fwd_packets            int64  
 2   total_backward_packets       int64  
 3   total_length_of_fwd_packets  int64  
 4   total_length_of_bwd_packets  int64  
 5   fwd_packet_length_max        int64  
 6   fwd_packet_length_min        int64  
 7   fwd_packet_length_mean       float64
 8   fwd_packet_length_std        float64
 9   bwd_packet_length_max        int64  
 10  bwd_packet_length_min        int64  
 11  bwd_packet_length_mean       float64
 12  bwd_packet_length_std        float64
 13  flow_bytes/s                 float64
 14  flow_packets/s               float64
 15  flow_iat_mean                float64
 16  flow_iat_std          

In [ ]:
# df1 = pd.read_csv("D:/IoC-Free IDS using ML and NLP/data/processed/cicids2017_cleaned_standardized.csv")
# df1.isnull().sum()

Unnamed: 0                     0
flow_duration                  0
total_fwd_packets              0
total_backward_packets         0
total_length_of_fwd_packets    0
                              ..
idle_mean                      0
idle_std                       0
idle_max                       0
idle_min                       0
label                          0
Length: 71, dtype: int64